# Reading Code_Saturne `.case` files using pyvista
In this tutorial, Code_Saturne `.case` files which are typically processed in paraview are processed using pyvista. In this tutorial the dataset is extracted and then plotted.

In [1]:
import pyvista as pv

# .case file name
fn = 'case/postprocessing/RESULTS_FLUID_DOMAIN.case'

# Two ways of reading .case file
# Under the hood get_reader detects the file extension and 
# chooses the correct reader
reader1 = pv.get_reader(fn)
reader2 = pv.EnSightReader(fn)

print(reader1)
print(reader2)

EnSightReader('case/postprocessing/RESULTS_FLUID_DOMAIN.case')
EnSightReader('case/postprocessing/RESULTS_FLUID_DOMAIN.case')


In [2]:
# Output available times
reader1.time_values


[25.0,
 50.0,
 75.0,
 100.0,
 125.0,
 150.0,
 175.0,
 200.0,
 225.0,
 250.0,
 275.0,
 300.0]

In [3]:
# Set active time and read dataset
reader1.set_active_time_value(300)
data_blocks = reader1.read()

# results in multi-block dataset with one block: 'Fluid domain'
print(data_blocks)
print(data_blocks.keys())

MultiBlock (0x70b6f6bb96c0)
  N Blocks    1
  X Bounds    -1.000, 1.000
  Y Bounds    -1.000, 1.000
  Z Bounds    0.000, 20.000
['Fluid domain']


In [4]:
# Access the unstructured dataset by indexing like a dict
data = data_blocks['Fluid domain']
print(data)

UnstructuredGrid (0x70b6f6bb9c60)
  N Cells:    3140920
  N Points:   3172091
  X Bounds:   -1.000e+00, 1.000e+00
  Y Bounds:   -1.000e+00, 1.000e+00
  Z Bounds:   0.000e+00, 2.000e+01
  N Arrays:   10


In [5]:
# View cell data (think like paraview)
data.cell_data

pyvista DataSetAttributes
Association     : CELL
Active Scalars  : mpi_rank_id
Active Vectors  : Velocity
Active Texture  : None
Active Normals  : None
Contains arrays :
    mpi_rank_id             float32    (3140920,)           SCALARS
    Velocity                float32    (3140920, 3)         VECTORS
    Pressure                float32    (3140920,)
    TurbVisc                float32    (3140920,)
    CourantNb               float32    (3140920,)
    FourierNb               float32    (3140920,)
    total_pressure          float32    (3140920,)
    U_mean                  float32    (3140920, 3)
    UU_mean                 float32    (3140920, 6)
    P_mean                  float32    (3140920,)

In [6]:
# Common filter in paraview
data = data.cell_data_to_point_data() # data.ctp()

# Show point data
data.point_data

pyvista DataSetAttributes
Association     : POINT
Active Scalars  : mpi_rank_id
Active Vectors  : Velocity
Active Texture  : None
Active Normals  : None
Contains arrays :
    mpi_rank_id             float32    (3172091,)           SCALARS
    Velocity                float32    (3172091, 3)         VECTORS
    Pressure                float32    (3172091,)
    TurbVisc                float32    (3172091,)
    CourantNb               float32    (3172091,)
    FourierNb               float32    (3172091,)
    total_pressure          float32    (3172091,)
    U_mean                  float32    (3172091, 3)
    UU_mean                 float32    (3172091, 6)
    P_mean                  float32    (3172091,)

In [7]:
# Creater plotter
p = pv.Plotter(window_size=(500,500))

# Add dataset to plotter displaying velocity with the bwr colormap
# To access components of a vector use keyword argument `component=0` etc.
p.add_mesh(data,
           scalars='Velocity',
           cmap='bwr')

# Set view and add axes
p.view_xy()
p.add_axes()
p.show()

Widget(value='<iframe src="http://localhost:45319/index.html?ui=P_0x70b6f6d14950_0&reconnect=auto" class="pyvi…

In [8]:
# Same as before but create slice (like the slice filter in paraview)
p = pv.Plotter(window_size=(500,500))

plane_slice = data.slice(origin=(0,0,0),
                         normal='x')

p.add_mesh(plane_slice,
           cmap='bwr',
           scalars='Velocity')
p.view_zy()
p.add_axes()
p.show()


Widget(value='<iframe src="http://localhost:45319/index.html?ui=P_0x70b6e0186cd0_1&reconnect=auto" class="pyvi…